## PyTorch/XLA ResNet18/CIFAR10 (GPU or TPU)

### [RUNME] Install Colab compatible PyTorch/XLA wheels and dependencies



In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 152.9 MB 28 kB/s 
     |████████████████████████████████| 57 kB 2.9 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.306 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


### If you're using GPU with this colab notebook, run the below commented code to install GPU compatible PyTorch wheel and dependencies

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

40

In [ ]:
#!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/cuda/112/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl --force-reinstall 

### Only run the below commented cell if you would like a nightly release

In [ ]:
# VERSION = "1.11"  #@param ["1.11", "nightly", "20220315"]  # or YYYYMMDD format
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION
# import os 
# os.environ['LD_LIBRARY_PATH']='/usr/local/lib'
# !echo $LD_LIBRARY_PATH

# !sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
# !sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
# !sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

# !ldconfig
# !ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

In [ ]:
# PyTorch/XLA GPU Setup (only if GPU runtime)
import os
# if os.environ.get('COLAB_GPU', '0') == '1':
#   os.environ['GPU_NUM_DEVICES'] = '1'
#   os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda/'

In [ ]:
# Downloading the clevr dataset takes about 5-7 min
!wget https://dl.fbaipublicfiles.com/clevr/CLEVR_v1.0.zip
# unzip the file takes about 3 min
!unzip -q CLEVR_v1.0.zip

--2022-04-27 09:05:55--  https://dl.fbaipublicfiles.com/clevr/CLEVR_v1.0.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19021600724 (18G) [application/zip]
Saving to: ‘CLEVR_v1.0.zip’

CLEVR_v1.0.zip      100%[===================>]  17.71G  54.8MB/s    in 5m 29s  

2022-04-27 09:11:25 (55.1 MB/s) - ‘CLEVR_v1.0.zip’ saved [19021600724/19021600724]



In [ ]:
# install the transformer library
!pip -q install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 895 kB 72.6 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 6.6 MB 70.8 MB/s 


In [ ]:
# Lets explore the dataset
import json
#training_questions = json.load(open("CLEVR_v1.0/questions/CLEVR_train_questions.json"))
#valid_questions = json.load(open("CLEVR_v1.0/questions/CLEVR_val_questions.json"))

In [ ]:
'''
questions - contains all the questions,
image_index - tells us for which image the question is for
question_index - is a unique id for the question
program - idk
image_filename - gives the image file name for which this question is asked
question_family_index - Question family (read the CLEVR paper for details)
answer - this is the answer to the question 
        There is something fishy with the answers only 28 unique answers
        in all the training data


'''

from collections import defaultdict
temp_dictionary = defaultdict(int)
# temp set should contain all the different answers that are there in the dataset
temp_set = set()

#for index,i in enumerate(training_questions["questions"]):
    # temp_dictionary[i["answer"]] += 1
    #temp_set.add(i["answer"])
    # if index < 5:
    #     #print(i.keys())
    #     print(i["image_filename"])
    #     print(i["answer"])
    # else:
    #     break


#print(len(temp_set))

# for index,i in enumerate(valid_questions["questions"]):
#     # temp_dictionary[i["answer"]] += 1
#     # temp_set.add(i["answer"])
#     if index < 5:
#         #print(i.keys())
#         print(i["question"])
#         print(i["answer"])
#     else:
#         break


# print(len(temp_set))

In [ ]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import torch, os, json
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from PIL import Image
import torchvision.models as models

torch.manual_seed(42)

In [ ]:
torch.__version__

'1.11.0+cu113'

In [ ]:

# Defining the dataloader 
class CLEVER(torch.utils.data.Dataset):
    def __init__(self,categories = None,folder = 'CLEVR_v1.0/', split = "train",transformation = None):
        self.json_dir = os.path.join(folder,'questions',f"CLEVR_{split}_questions.json")
        self.image_dir = os.path.join(folder,'images',split)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.categories = categories
        self.data = []
        self.cat2id = {i:index for index,i in enumerate(categories)}
        self.id2cat = {index:i for index,i in enumerate(categories)}
        self.transformation = transformation
        json_file = json.load(open(self.json_dir))
        
        for index,i in enumerate(json_file["questions"]):
            text = i["question"]
            answer = self.cat2id[i["answer"]]
            self.data.append((i["image_index"],i["image_filename"],text,answer))
        
    def __getitem__(self,index):
        image_path = os.path.join(self.image_dir,self.data[index][1])
        image = Image.open(image_path).convert('RGB')
        if self.transformation != None:
            image = self.transformation(image)
        text = self.data[index][2]
        encoded_text = self.tokenizer.encode_plus(
                text, add_special_tokens = True, truncation = True, 
                max_length = 256, padding = 'max_length',
                return_attention_mask = True,
                return_tensors = 'pt')
        text = encoded_text['input_ids'][0]
        attention_mask = encoded_text['attention_mask'][0]
        answer = self.data[index][3]
        return image,text,attention_mask,answer

    def __len__(self):
        return len(self.data)



In [ ]:
#categories = list(sorted(list(temp_set)))
categories = ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'blue', 'brown', 'cube', 'cyan', 'cylinder', 'gray', 'green', 'large', 'metal', 'no', 'purple', 'red', 'rubber', 'small', 'sphere', 'yellow', 'yes']
print(categories)

# norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# transform_train = transforms.Compose([
#     transforms.ToTensor(),
#     norm,
# ])

#train_data = CLEVER(categories = categories,split = 'train',transformation = transform_train)
# valid_data = CLEVER(categories = categories,split = 'val')

['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'blue', 'brown', 'cube', 'cyan', 'cylinder', 'gray', 'green', 'large', 'metal', 'no', 'purple', 'red', 'rubber', 'small', 'sphere', 'yellow', 'yes']


In [ ]:

# train_loader = torch.utils.data.DataLoader(
#         train_data,
#         batch_size=FLAGS['batch_size'],
#         num_workers=FLAGS['num_workers'],
#         drop_last=True)

# model = custom_model(categories)
# optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=5e-4)
# loss_fn = nn.CrossEntropyLoss()

# def train_loop_fn(loader):
#     model.train()
    
#     for x, (img,text,mask,label) in enumerate(loader):
#         optimizer.zero_grad()
#         output = model(img,text,mask)
#         loss = loss_fn(output, label)
#         print(loss)
#         loss.backward()
#         optimizer.step()

# # Train and eval loops
# accuracy = 0.0
# data, pred, target = None, None, None
# for epoch in range(1, FLAGS['num_epochs'] + 1):
#     print(epoch)
#     train_loop_fn(train_loader)


In [ ]:
from transformers import BertForSequenceClassification, BertConfig

# # Loading BERT...
# num_categories = len(categories)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
#     num_labels = num_categories,  output_attentions = False, 
#     output_hidden_states = False)

In [ ]:
class custom_model(nn.Module):
    def __init__(self):
        super(custom_model, self).__init__()

        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
        num_labels = 28,  output_attentions = False, 
        output_hidden_states = False)

        for param in self.bert.parameters():
            param.requires_grad = False

        self.cnn = models.efficientnet_b0(pretrained = True)
        for param in self.cnn.parameters():
            param.requires_grad = False

        self.linear1 = torch.nn.Linear(1028,500)
        self.linear2 = torch.nn.Linear(500,28)

    def forward(self,image_data,text_data,text_masks):
        x1 = self.cnn(image_data)
        x2 = self.bert(text_data,text_masks)
        x = torch.cat((x1,x2.logits),dim = 1)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [ ]:
SERIAL_EXEC = xmp.MpSerialExecutor()
# Only instantiate model weights once in memory.
WRAPPED_MODEL = xmp.MpModelWrapper(custom_model())

def train_custom_model():
    torch.manual_seed(1)

    def get_dataset():
        norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            norm,
        ])
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            norm,
        ])
        train_dataset = CLEVER(categories = categories,split = 'train',transformation = transform_train)
        test_dataset = CLEVER(categories = categories,split = 'val',transformation = transform_test)
        
        return train_dataset, test_dataset
    
    # Using the serial executor avoids multiple processes
    # to download the same data.
    train_dataset, test_dataset = SERIAL_EXEC.run(get_dataset)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
            train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True)
    train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=FLAGS['batch_size'],
            sampler=train_sampler,
            num_workers=FLAGS['num_workers'],
            drop_last=True)
    test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=FLAGS['batch_size'],
            shuffle=False,
            num_workers=FLAGS['num_workers'],
            drop_last=True)

    # Scale learning rate to num cores
    learning_rate = FLAGS['learning_rate'] * xm.xrt_world_size()

    # Get loss function, optimizer, and model
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=5e-4)
    loss_fn = nn.CrossEntropyLoss()

    def train_loop_fn(loader):
        tracker = xm.RateTracker()
        model.train()
        for x, (img,text,mask,label) in enumerate(loader):
            optimizer.zero_grad()
            output = model(img,text,mask)
            loss = loss_fn(output, label)
            #print(loss)
            loss.backward()
            xm.optimizer_step(optimizer)
            tracker.add(FLAGS['batch_size'])
            if x % FLAGS['log_steps'] == 0:
                print('[xla:{}]({}) Loss={:.5f} Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
                    xm.get_ordinal(), x, loss.item(), tracker.rate(),
                    tracker.global_rate(), time.asctime()), flush=True)

    def test_loop_fn(loader):
        total_samples = 0
        correct = 0
        model.eval()
        data, pred, target = None, None, None
        for x, (img,text,mask,label) in enumerate(loader):
            output = model(img,text,mask)
            pred = output.max(1, keepdim=True)[1]
            #print(pred)
            correct += pred.eq(label.view_as(pred)).sum().item()
            total_samples += data.size()[0]

            accuracy = 100.0 * correct / total_samples
            print('[xla:{}] Accuracy={:.2f}%'.format(
                xm.get_ordinal(), accuracy), flush=True)
        return accuracy, data, pred, target

    # Train and eval loops
    accuracy = 0.0
    data, pred, target = None, None, None
    for epoch in range(1, FLAGS['num_epochs'] + 1):
        para_loader = pl.ParallelLoader(train_loader, [device])
        train_loop_fn(para_loader.per_device_loader(device))
        xm.master_print("Finished training epoch {}".format(epoch))

        #para_loader = pl.ParallelLoader(test_loader, [device])
        #accuracy, data, pred, target  = test_loop_fn(para_loader.per_device_loader(device))
        if FLAGS['metrics_debug']:
            xm.master_print(met.metrics_report(), flush=True)

    return accuracy, data, pred, target
  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
os.environ.get('TPU_NAME', None)

'grpc://10.37.141.2:8470'

In [ ]:
# Define Parameters
FLAGS = {}
FLAGS['batch_size'] = 128
FLAGS['num_workers'] = 8
FLAGS['learning_rate'] = 0.001 #0.02
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 20
FLAGS['num_cores'] = 8 # 8 for tpu 1 for gpu
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False

In [ ]:
def _mp_fn(rank, flags): # Start training processes
    global FLAGS
    FLAGS = flags
    torch.set_default_tensor_type('torch.FloatTensor')
    accuracy, data, pred, target = train_custom_model()


xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS['num_cores'],
        start_method='fork')

KeyboardInterrupt: ignored

# Trying an alternate way


In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_train = transforms.Compose([
    transforms.ToTensor(),
    norm,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    norm,
])

train_dataset = CLEVER(categories = categories,split = 'train',transformation = transform_train)
test_dataset = CLEVER(categories = categories,split = 'val',transformation = transform_test)


train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=FLAGS['batch_size'],
        num_workers=FLAGS['num_workers'],
        drop_last=True)
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=FLAGS['batch_size'],
        shuffle=False,
        num_workers=FLAGS['num_workers'],
        drop_last=True)



In [ ]:
flags = {
    "num_epochs":10,
    "batch_size":32,
    "num_workers":8,
}


def _mp_fn(index,flags):
    torch.manual_seed(42)
    device = xm.xla_device()

    norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        norm,
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        norm,
    ])

    if not xm.is_master_ordinal():
        xm.rendezvous('download_only_once')
    
    train_dataset = CLEVER(categories = categories,split = 'train',transformation = transform_train)
    test_dataset = CLEVER(categories = categories,split = 'val',transformation = transform_test)

    if xm.is_master_ordinal():
        xm.rendezvous('download_only_once')
    
    train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)

    test_sampler = torch.utils.data.distributed.DistributedSampler(
    test_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=flags['batch_size'],
        sampler=train_sampler,
        num_workers=flags['num_workers'],
        drop_last=True)
    
    test_loader = torch.utils.data.DataLoader(
            test_dataset,
            sampler=test_sampler,
            batch_size=flags['batch_size'],
            shuffle=False,
            num_workers=flags['num_workers'],
            drop_last=True)

    model = custom_model().to(device).train()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = 0.001)
    for epoch in range(flags["num_epochs"]):
        para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
        for x, (img,text,mask,label) in enumerate(para_train_loader):
            optimizer.zero_grad()
            output = model(img,text,mask)
            loss = loss_fn(output, label)
            print(loss)
            loss.backward()
            xm.optimizer_step(optimizer)

if __name__ == "__main__":
    xmp.spawn(_mp_fn,args=(flags,), nprocs = 8, start_method = "fork")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

ProcessExitedException: ignored